<a href="https://colab.research.google.com/github/Vaycold/pytorch_tutorial/blob/main/%2306.%EB%AA%A8%EB%8D%B8%20%EB%A7%A4%EA%B0%9C%EB%B3%80%EC%88%98%20%EC%B5%9C%EC%A0%81%ED%99%94%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 기본 코드

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


### 하이퍼파라미터

    : 하이퍼파라미터는 모델 최적화 과정을 제어할 수 있는 조절 가능한 매개변수
    : 서로 다른 하이퍼파라미터 값은 모델 학습과 수렴율에 영향을 미침
    : 학습 시 에는 다음과 같은 하이퍼파라미터를 정의
     - epoch         : 데이터셋을 반복하는 횟수
     - batch size    : 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플의 수
     - learning rate : 각 배치/에폭에서 모델의 매개변수를 조절하는 비율.

In [2]:
epochs = 5
batch_size = 64
learning_rate = 1e-3

### 최적화 단계

    : 하이퍼파라미터를 설정한 뒤에는 최적화 단계를 통해 모델 학습하고 최적화할 수 있음.
    : 최적화 단계의 각 반복을 에폭이라 부름
    : 하나의 에폭은 다음 2 부분으로 구성
     - 학습단계.       - 학습용 데이터셋을 반복하고 최적의 매개변수로 수렴
     - 검증/테스트 단계 - 모델 성능이 개선되고 있는 지를 확인하기 위해 테스트 데이터셋을 반복

    

#### 학습단계

    : 손실함수
     - 학습용 데이터를 제공하면 학습되지 않은 신경망은 정답을 제공하지 않을 확률이 높음
     - 손실함수는 획득한 결과와 실제 값 사이의 틀린 정도를 측정하며, 학습 중에 이 값을 최소화하려고 함
     - 일반적인 손실 함수에는 회귀문제에 사용하는 nn.MSELoss, 분류에 사용하는 nn.NLLLoss, nn.CrossEntropyLoss

In [3]:
# 손실함수 초기화
loss_fn = nn.CrossEntropyLoss()

    : 옵티마이저
     - 최적화는 각 학습단계에서 모델의 오류를 줄이기 위해 모델 매개변수를 조정하는 과정
     - 모든 최적화 절차는 optimizer 객체에 캡슐화됨.

In [4]:
# 학습하려는 모델의 매개변수와 학습률 하이퍼파라미터를 등록하여 옵티마이저 초기화
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

    : 학습단계에서 최적화는 3단계로 이루어짐
     - 1. optimizer.zero_grad()를 호출하여 모델 매개변수의 변화도를 재설정
       > 기본적으로 변화도는 더해지기 때문에 중복계산을 막기위해 반복할 때마다 명시적으로 0으로 설정
      - 2. loss.backward()를 호출하여 예측 손실을 역전파
      - 3. 변화도를 계산한 뒤 optimizer.step()을 호출하여 역전파 단계에서 수집된 변화도로 매개변수 조정

In [7]:
# 최적화 코드를 반복하여 수행하는 train_loop와 테스트 데이터로 모델의 성능을 측정하는 test_loop를 정의
def train_loop(dataloader, model, loss_fn, optimizer) :
    size = len(dataloader.dataset) 

    for batch, (x,y) in enumerate(dataloader) :
        # 예측(prediction) 과 손실(loss) 계산
        pred = model(x)
        loss = loss_fn(pred,y)
        
        #역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0 :
            loss, current = loss.item(), batch*len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn) :
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0,0

    with torch.no_grad() :
        for x,y in dataloader :
            pred = model(x)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

epochs = 10
for t in range(epochs) : 
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print('Done!')

Epoch 1
-------------------------------
loss: 2.302581  [    0/60000]
loss: 2.300091  [ 6400/60000]
loss: 2.297679  [12800/60000]
loss: 2.291710  [19200/60000]
loss: 2.281147  [25600/60000]
loss: 2.283646  [32000/60000]
loss: 2.265268  [38400/60000]
loss: 2.259744  [44800/60000]
loss: 2.262137  [51200/60000]
loss: 2.242863  [57600/60000]
Test Error: 
 Accuracy: 39.9%, Avg loss: 2.245701 

Epoch 2
-------------------------------
loss: 2.249041  [    0/60000]
loss: 2.227043  [ 6400/60000]
loss: 2.221402  [12800/60000]
loss: 2.233422  [19200/60000]
loss: 2.188790  [25600/60000]
loss: 2.208787  [32000/60000]
loss: 2.175480  [38400/60000]
loss: 2.169506  [44800/60000]
loss: 2.180854  [51200/60000]
loss: 2.148702  [57600/60000]
Test Error: 
 Accuracy: 45.1%, Avg loss: 2.149936 

Epoch 3
-------------------------------
loss: 2.149366  [    0/60000]
loss: 2.104615  [ 6400/60000]
loss: 2.097608  [12800/60000]
loss: 2.138886  [19200/60000]
loss: 2.046613  [25600/60000]
loss: 2.091283  [32000/600